## 1.3 Input Layer Stabilization

Input Layer Stabilization은 가장 간단하지만 파워풀한 정규화 방법입니다. 이전 실습과 동일한 부분들은 일단 동일하게 적용해줍니다. 

In [ ]:
import math
import random 

import tensorflow as tf
import numpy as np 

seed = 1
random.seed(seed)
np.random.seed(seed=seed)
tf.random.set_random_seed(seed)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape([-1, 28 * 28])
x_test = x_test.reshape([-1, 28 * 28])

m = np.random.randint(0, high=60000, size=1100, dtype=np.int64)
x_train = x_train[m]
y_train = y_train[m]

i = np.arange(1100)
np.random.shuffle(i)
x_train = x_train[i]
y_train = y_train[i]

x_valid = x_train[:100]
y_valid = y_train[:100]

x_train = x_train[100:]
y_train = y_train[100:]

이제 여러 Input Layer Stabilization 방식 중 Standardization을 적용합니다.

In [ ]:
mean, std = np.mean(x_train), np.std(x_train)
x_train = (x_train.astype(np.float64)- mean) / std
x_valid = (x_valid.astype(np.float64)- mean) / std
x_test = (x_test.astype(np.float64) - mean) / std

Valid set과 Test set은 트레이닝할 때 주어지지 않는다고 가정해야 하므로, Training 데이터만을 이용해 평균과 표준편차를 계산하고 validation, test 셋도 동일하게 표준화 해줍니다. (수업시간에 배웠듯, RGB 채널이 있는 데이터의 경우에는 RGB 채널별로 표준화 하는 것이 일반적입니다.)

이제 네트워크를 이전 실습과 동일하게 초기화 합니다. 

In [ ]:
x = tf.placeholder(tf.float32, [None, 28 * 28])
y = tf.placeholder(tf.int32, [None])

n_units = [28 * 28, 512, 512, 10]

weights, biases = [], []
for i, (n_in, n_out) in enumerate(zip(n_units[:-1], n_units[1:])):
    stddev = math.sqrt(2 / n_in) # Kaiming He Initialization
    weight = tf.Variable(tf.random.truncated_normal([n_in, n_out], mean=0, stddev=stddev))
    bias = tf.Variable(tf.zeros([n_out]))
    weights.append(weight)
    biases.append(bias)
    
layer = x 
for i, (weight, bias) in enumerate(zip(weights, biases)):
    layer = tf.matmul(layer, weight) + bias
    if i < len(weights) - 1:
        layer = tf.nn.tanh(layer)        
y_hat = layer

y_hot = tf.one_hot(y, 10)
costs = tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y_hot, logits=y_hat)
cross_entropy_loss = tf.reduce_mean(costs)
loss = cross_entropy_loss 

accuracy = tf.count_nonzero(
        tf.cast(tf.equal(tf.argmax(y_hot, 1), tf.argmax(y_hat, 1)),
                tf.int64)) / tf.cast(tf.shape(y_hot)[0], tf.int64)

extra_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_ops):
    optimizer = tf.train.AdamOptimizer(1e-3)
    train_op = optimizer.minimize(loss)
    
gpu_options = tf.GPUOptions()
gpu_options.allow_growth = True
session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
session.run(tf.global_variables_initializer())

학습을 진행합니다. 총 1000번의 Epoch을 수행합니다. 총 100 Epoch 동안 성능 개선이 이뤄지지 않으면 Early Stop 합니다. 

In [ ]:
max_valid_epoch_idx = 0
max_valid_accuracy = 0.0
final_test_accuracy = 0.0
for epoch_idx in range(1, 1000 + 1):
    session.run(
            train_op,
            feed_dict={
                x: x_train,
                y: y_train
            })
    
    if epoch_idx % 10 == 0:
        train_loss_value, train_accuracy_value = session.run(
            [loss, accuracy],
            feed_dict={
                x: x_train,
                y: y_train
            })
        
        valid_loss_value, valid_accuracy_value = session.run(
            [loss, accuracy],
            feed_dict={
                x: x_valid,
                y: y_valid
            })
            
        test_loss_value, test_accuracy_value = session.run(
            [loss, accuracy],
            feed_dict={
                x: x_test,
                y: y_test
            })

        print(epoch_idx, '%.4f' % train_loss_value, '%.4f' % valid_loss_value, '%.4f' % test_loss_value, '%.4f' % train_accuracy_value, '%.4f' % valid_accuracy_value, '%.4f' % test_accuracy_value)
        
        if max_valid_accuracy < valid_accuracy_value:
            max_valid_accuracy = valid_accuracy_value 
            max_valid_epoch_idx = epoch_idx
            final_test_accuracy = test_accuracy_value
            
    # Early Stop
    if max_valid_epoch_idx + 100 < epoch_idx:
        break
        
print(final_test_accuracy)

우리는 이번 실습에서 정말 간단한 표준화 만으로도 88.05% -> 90.55% 로 성능이 2.5% 향상한 것을 확인할 수 있었습니다.

### 연습 문제

Q1. 표준화를 zero-one Normalization으로 바꿔보고 성능을 측정해보세요. 
(힌트는 [여기](01_03_input_layer_stabilization_Q1_hint.txt)에서 확인하실 수 있습니다.)
(정답은 [여기](01_03_input_layer_stabilization_Q1_answer.txt)에서 확인하실 수 있습니다.)

주의사항! 코드를 수정한 이후에는 Kernel > Restart & Run All 을 통해 네트워크를 처음부터 다시 학습시켜 주세요. 

### 다음 실습

다음 [실습](01_04_loss_panelty.ipynb)에서는 Loss Panelty를 통해 딥 모델을 정규화하는 방법을 알아보고자 합니다.